# make lookup table from whole row data
-----
・use pubchempy and CASRN, get need property from PubChem

In [185]:
import os
import sys

current_dir = os.getcwd()
parent_parent_dir = os.path.dirname(os.path.dirname(current_dir))
src_dir = os.path.join(parent_parent_dir, 'src')
sys.path.append(src_dir)

from prep import prep_pubchem_bycas
from prep import for_lookup
from util import pickle_load
from util import pickle_dump
from util import robust_z

import sqlite3
import time
import pandas as pd
from tqdm import tqdm
import pubchempy as pcp
import collections
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem
import numpy as np
from scipy.stats import norm
from sklearn.preprocessing import robust_scale

ImportError: cannot import name 'for_lookup' from 'prep' (/workspace/HDD1/user/ohto/3-GA/240625_GA_for_TOXPRED/src/prep.py)

In [ ]:
# use pubchempy for gathering data (canonical smiles, xlogp, and TPSA) for GA
# before using pubchempy, gather all CAS-RN in all data 

cas = []
for test_num in ["0901", "0902", "0904", "0905"]:
    for lig in ["ago", "anta"]:
        for file_name in ["for_GA", "validation"]:
            df = pd.read_csv(f"../../data/processed/{test_num}_{lig}/{file_name}.tsv", sep="\t", header=None)
            df = df.dropna()
            for i in range(len(df)):
                cas.append(df.iloc[i,0])

for test_num in ["0701", "0702", "0907", "1001", "1002"]:
    for file_name in ["for_GA", "validation"]:
        df = pd.read_csv(f"../../data/processed/{test_num}/{file_name}.tsv", sep="\t", header=None)
        df = df.dropna()
        for i in range(len(df)):
            cas.append(df.iloc[i,0])

cas = list(set(cas))

In [ ]:
property = ['CanonicalSMILES', 'XLogP', 'TPSA']
all_data = prep_pubchem_bycas(cas, property)

100%|██████████| 14598/14598 [11:28:15<00:00,  2.83s/it]  


In [ ]:
all_data_na = all_data[0].dropna()
all_data_na.to_csv("../../data/processed/other/all_pubchem_data.tsv", sep="\t", header=None, index=False)

In [ ]:
# error means all CASRN that cannot get full data from pubchem
error = set(all_data[1]) | set(all_cas - all_cas_na)
pickle_dump(error, "../../data/processed/other/all_pubchem_error.pickle")

In [ ]:
# select pubchem data if a CASRN has multiple CIDs
all_data_na = all_data_na.reset_index()
all_data_na = all_data_na.drop(columns=["index"])

all_data_dict = dict()
for i in tqdm(range(len(all_data_na))):
    cas = all_data_na["CAS"][i]
    if cas not in all_data_dict.keys():
        all_data_dict[cas] = []
        for n in range(len(all_data_na.iloc[i])):
            all_data_dict[cas].append(all_data_na.iloc[i,n])
    else:
        continue

100%|██████████| 11412/11412 [00:06<00:00, 1895.22it/s]


In [ ]:
df = pd.DataFrame.from_dict(all_data_dict, orient='index', columns=["CID","CAS","CanonicalSmiles","xlogp","tpsa"]).reset_index().drop(columns=["index"])
df.to_csv("../../data/processed/other/all_pubchem_data.tsv",sep="\t",index=False)

In [ ]:
lookup_whole = for_lookup(all_data_na)

100%|██████████| 13687/13687 [36:18<00:00,  6.28it/s] 


In [ ]:
pickle_dump(lookup_whole, "../../data/processed/other/lookup_whole.pickle")

In [ ]:
# make for_GA dataset 
for test_num in ["0901", "0902", "0904", "0905"]:
    for lig in ["ago", "anta"]:
        for file_name in ["for_GA"]:
            df = pd.read_csv(f"../../data/processed/{test_num}_{lig}/{file_name}.tsv", sep="\t", header=None)
            df = df.dropna()
            cas = []
            for i in range(len(df)):
                cas.append(df.iloc[i,0])
            use_data = all_data_na[all_data_na["CAS"].isin(cas)].reset_index().drop(columns=["index"])
            lookup = for_lookup(use_data)
            pickle_dump(lookup, f"../../data/processed/{test_num}_{lig}/{file_name}_lookup.pickle")

for test_num in ["0701", "0702", "0907", "1001", "1002"]:
    for file_name in ["for_GA"]:
        df = pd.read_csv(f"../../data/processed/{test_num}/{file_name}.tsv", sep="\t", header=None)
        df = df.dropna()
        cas = []
        for i in range(len(df)):
            cas.append(df.iloc[i,0])
        use_data = all_data_na[all_data_na["CAS"].isin(cas)].reset_index().drop(columns=["index"])
        lookup = for_lookup(use_data)
        pickle_dump(lookup, f"../../data/processed/{test_num}/{file_name}_lookup.pickle")

100%|██████████| 431/431 [00:02<00:00, 203.02it/s]


In [ ]:
# if compound in validation dataset cannot make lookup, collect data from text in JACVAM
correct_cas = set(all_data_na["CAS"])

all_val_error_cas = set()
for test_num in ["0901", "0902", "0904", "0905"]:
    for lig in ["ago", "anta"]:
        val = pd.read_csv(f"../../data/processed/{test_num}_{lig}/validation.tsv", sep="\t", header=None)
        val_cas = set(val.iloc[:,0])
        val_error_cas = val_cas - correct_cas
        all_val_error_cas = all_val_error_cas | val_error_cas
        print(test_num, lig, val_error_cas)

for test_num in ["0701", "0702", "0907", "1001", "1002"]:
    val = pd.read_csv(f"../../data/processed/{test_num}/validation.tsv", sep="\t", header=None)
    val_cas = set(val.iloc[:,0])
    val_error_cas = val_cas - correct_cas
    all_val_error_cas = all_val_error_cas | val_error_cas
    print(test_num, val_error_cas)

0901 ago {'50-41-9'}
0901 anta {'82640-04-8'}
0902 ago {'50-41-9', '57-30-7', '1461-22-9'}
0902 anta {'50-41-9', '82640-04-8'}
0904 ago set()
0904 anta set()
0905 ago set()
0905 anta set()
0701 {'152-11-4', '7789-12-0', '1910-42-5', '73791-47-6', '318-98-9', '10043-35-3', '7447-40-7', '7681-49-4', '1330-20-7', '51-42-3', '614-39-1', '1327-53-3', '76-87-9', '7487-94-7', '7647-14-5', '8007-59-8', '554-13-2', '7784-46-5', '151-50-8', '7446-18-6', '10108-64-2', '7758-99-8', '13410-01-0', '62-76-0', '549-18-8'}
0702 {'557-05-1', '7778-80-5', '107-64-2', '25646-77-9', '3926-62-3', '10361-37-2', '1314-13-2', '124-07-02', '917-61-3', '7758-98-7', '866-84-2', '12125-02-9', '7779-90-0'}
0907 {'2943-75-1', '84852-15-3'}
1001 {'82385-42-0', '10043-35-3', '54-21-7', '147-24-0', '69-57-8', '115-09-3', '7447-41-8'}
1002 {'10043-35-3', '147-24-0', '34381-68-5'}


In [ ]:
property = ['CanonicalSMILES', 'XLogP', 'TPSA']

for test_num in ["0901", "0902", "0904", "0905"]:
    for lig in ["ago", "anta"]:
        val = pd.read_csv(f"../../data/processed/{test_num}_{lig}/validation.tsv", sep="\t", header=None)
        tsv = []
        for i in tqdm(range(len(val))):
            if val.iloc[i,0] in all_val_error_cas:
                time.sleep(2)
                data = pcp.get_properties(property, val.iloc[i,0], "name", as_dataframe=True)
                data = data.reset_index()
                col = []
                col.append(val.iloc[i,0])
                col.append(val.iloc[i,1])
                try:col.append(data["CanonicalSMILES"][0])
                except:col.append("###")
                try:col.append(data["XLogP"][0])
                except:col.append("###")
                try:col.append(data["TPSA"][0])
                except:col.append("###")
                tsv.append(col)
            else:
                col = []
                col.append(val.iloc[i,0])
                col.append(val.iloc[i,1])
                for n in range(2, len(all_data_dict[val.iloc[i,0]])):
                    col.append(all_data_dict[val.iloc[i,0]][n])
                tsv.append(col)
        pd.DataFrame(tsv).to_csv(f"../../data/processed/{test_num}_{lig}/validation_pubchem.tsv", sep="\t", header=None, index=False)

for test_num in ["0701", "0702", "0907", "1001", "1002"]:
    val = pd.read_csv(f"../../data/processed/{test_num}/validation.tsv", sep="\t", header=None)
    tsv = []
    for i in tqdm(range(len(val))):
        if val.iloc[i,0] in ["8007-59-8", "1327-53-3", "1330-20-7","84852-15-3"]:
            print(test_num, val.iloc[i,0])
        elif val.iloc[i,0] in all_val_error_cas:
            time.sleep(2)
            data = pcp.get_properties(property, val.iloc[i,0], "name", as_dataframe=True)
            data = data.reset_index()
            col = []
            col.append(val.iloc[i,0])
            col.append(val.iloc[i,1])
            try:col.append(data["CanonicalSMILES"][0])
            except:col.append("###")
            try:col.append(data["XLogP"][0])
            except:col.append("###")
            try:col.append(data["TPSA"][0])
            except:col.append("###")
            tsv.append(col)
        else:
            col = []
            col.append(val.iloc[i,0])
            col.append(val.iloc[i,1])
            for n in range(2, len(all_data_dict[val.iloc[i,0]])):
                col.append(all_data_dict[val.iloc[i,0]][n])
            tsv.append(col)
    pd.DataFrame(tsv).to_csv(f"../../data/processed/{test_num}/validation_pubchem.tsv", sep="\t", header=None, index=False)

 60%|█████▉    | 43/72 [00:32<00:26,  1.07it/s]

0701 8007-59-8


 67%|██████▋   | 48/72 [00:35<00:18,  1.27it/s]

0701 1327-53-3


 82%|████████▏ | 59/72 [00:55<00:25,  1.95s/it]

0701 1330-20-7


  0%|          | 0/26 [00:00<?, ?it/s]

0907 84852-15-3


100%|██████████| 16/16 [00:08<00:00,  1.78it/s]


## Replace "###" in test_name/validation_pubchem.tsv to correct data in Jacvam text

## get data about CAS-RN written below from PubChem
### 0701 8007-59-8(cas = 7681-52-9, xlogp is from rdkit) 1327-53-3(cas = CAS-1327-53-3, xlogp is from rdkit) 1330-20-7(cas = CAS-1330-20-7, xlogp is from rdkit)
### 0907 84852-15-3(cas = CAS-84852-15-3)

In [ ]:
for test_num in ["0901", "0902", "0904", "0905"]:
    for lig in ["ago", "anta"]:
        val = pd.read_csv(f"../../data/processed/{test_num}_{lig}/validation_pubchem.tsv", sep="\t", header=None)
        lookup = for_lookup(val)
        pickle_dump(lookup, f"../../data/processed/{test_num}_{lig}/validation_lookup.pickle")

100%|██████████| 9/9 [00:00<00:00, 11168.27it/s]


In [ ]:
for test_num in ["0701", "0702", "0907", "1001", "1002"]:
    val = pd.read_csv(f"../../data/processed/{test_num}/validation_pubchem.tsv", sep="\t", header=None)
    lookup = for_lookup(val)
    pickle_dump(lookup, f"../../data/processed/{test_num}/validation_lookup.pickle")

100%|██████████| 16/16 [00:00<00:00, 6116.37it/s]


## make cas and severity data only in whole dataset

In [ ]:
for test_num in ["0901", "0902", "0904", "0905"]:
    for lig in ["ago", "anta"]:
        file_name = "for_GA"
        df = pd.read_csv(f"../../data/processed/{test_num}_{lig}/{file_name}.tsv", sep="\t", header=None)
        df = df.dropna()
        cas = []
        for i in range(len(df)):
            cas.append(df.iloc[i,0])
        use_cas =  set(all_data_na[all_data_na["CAS"].isin(cas)].reset_index().drop(columns=["index"])["CAS"])
        test_name = test_num + "_" + lig
        cas_tox = pd.read_csv(f"../../data/processed/{test_name}/cas_sev.tsv", sep="\t", header=None)
        cas_tox_use = []
        for i in range(len(cas_tox)):
            if cas_tox.iloc[i,0] in use_cas:
                cas_tox_use.append([cas_tox.iloc[i,0], cas_tox.iloc[i,1]])
        pd.DataFrame(cas_tox_use).to_csv(f"../../data/processed/{test_name}/cas_sev_use.tsv", sep="\t", header=None, index=False)

In [ ]:
use_cas =  set(all_data_na[all_data_na["CAS"].isin(cas)].reset_index().drop(columns=["index"])["CAS"])
for test_num in ["0701", "0702", "0907", "1001", "1002"]:
    file_name = "for_GA"
    test_name = test_num
    df = pd.read_csv(f"../../data/processed/{test_num}/{file_name}.tsv", sep="\t", header=None)
    df = df.dropna()
    cas = []
    for i in range(len(df)):
        cas.append(df.iloc[i,0])
    use_cas =  set(all_data_na[all_data_na["CAS"].isin(cas)].reset_index().drop(columns=["index"])["CAS"])
    cas_tox = pd.read_csv(f"../../data/processed/{test_name}/cas_sev.tsv", sep="\t", header=None)
    cas_tox_use = []
    for i in range(len(cas_tox)):
        if cas_tox.iloc[i,0] in use_cas:
            cas_tox_use.append([cas_tox.iloc[i,0], cas_tox.iloc[i,1]])
    pd.DataFrame(cas_tox_use).to_csv(f"../../data/processed/{test_name}/cas_sev_use.tsv", sep="\t", header=None, index=False)

In [188]:
import pandas as pd
df = pd.read_csv("../../data/processed/other/all_pubchem_data.tsv", sep="\t") 

In [189]:
df

,CID,CAS,CanonicalSmiles,xlogp,tpsa
0,4870,346-18-9,CN1C(NC2=CC(=C(C=C2S1(=O)=O)S(=O)(=O)N)Cl)CSCC...,152.0,2.6
1,8430,120-40-1,CCCCCCCCCCCC(=O)N(CCO)CCO,60.8,3.5
2,33599,68515-49-1,CC(C)CCCCCCCOC(=O)C1=CC=CC=C1C(=O)OCCCCCCCC(C)C,52.6,10.6
3,5311051,25122-41-2,CC1CC2C3CCC4=CC(=O)C=CC4(C3(C(CC2(C1(C(=O)CCl)...,74.6,2.5
4,13254,831-82-3,C1=CC=C(C=C1)OC2=CC=C(C=C2)O,29.5,3.4
...,...,...,...,...,...
11407,10334118,697235-49-7,C1=CC=C(C(=C1)C(=O)O)NC(=O)CCC2=CC=C(C=C2)O,86.6,3.4
11408,22833,6358-64-1,COC1=CC(=C(C=C1N)OC)Cl,44.5,1.8
11409,17343,2539-17-5,COC1=C(C(=C(C(=C1Cl)Cl)Cl)Cl)O,29.5,4.4
11410,439260,51-41-2,C1=CC(=C(C=C1C(CN)O)O)O,86.7,-1.2


In [197]:
for test_num in ["0701", "0702", "0907", "1001", "1002"]:
    test_name = test_num
    cas_tox_use = pd.read_csv(f"../../data/processed/{test_name}/cas_sev_use.tsv", sep="\t", header=None)
    smiles = []
    for i in tqdm(range(len(cas_tox_use))):
        for n in range(len(df)):
            if cas_tox_use.iloc[i,0] == df.iloc[n,1]:
                smiles.append(df.iloc[n,2])
                break
    cas_tox_use.loc[:,3] = smiles
    pd.DataFrame(cas_tox_use).to_csv(f"../../data/processed/{test_name}/cas_sev_use.tsv", sep="\t", header=None, index=False)
    


  0%|          | 0/7046 [00:00<?, ?it/s]

100%|██████████| 431/431 [02:12<00:00,  3.26it/s]


In [198]:
for test_num in ["0901", "0902", "0904", "0905"]:
    for lig in ["ago", "anta"]:
        test_name = test_num + "_" + lig
        cas_tox_use = pd.read_csv(f"../../data/processed/{test_name}/cas_sev_use.tsv", sep="\t", header=None)
        smiles = []
        for i in range(len(cas_tox_use)):
            for n in range(len(df)):
                if cas_tox_use.iloc[i,0] == df.iloc[n,1]:
                    smiles.append(df.iloc[n,2])
        cas_tox_use.loc[:,3] = smiles
        pd.DataFrame(cas_tox_use).to_csv(f"../../data/processed/{test_name}/cas_sev_use.tsv", sep="\t", header=None, index=False)